In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [10]:
df = pd.read_csv('transactions_n100000.csv')

In [15]:
df.head(20)

,ticket_id,order_timestamp,location,item_name,item_count,lat,long
0,0x7901ee,2019-01-16 18:33:00,7,shake,2,41.794132,-88.010140
1,0x7901ee,2019-01-16 18:33:00,7,burger,2,41.794132,-88.010140
2,0x7901ee,2019-01-16 18:33:00,7,fries,2,41.794132,-88.010140
3,0x12b47f,2019-09-04 12:36:00,3,salad,1,41.884490,-87.627059
4,0x12b47f,2019-09-04 12:36:00,3,fries,1,41.884490,-87.627059
5,0x6d6979,2019-03-18 00:27:00,6,burger,2,41.784576,-87.607565
6,0x6d6979,2019-03-18 00:27:00,6,fries,2,41.784576,-87.607565
7,0x78dd1e,2019-09-22 00:10:00,2,shake,2,42.049306,-87.677606
8,0x78dd1e,2019-09-22 00:10:00,2,burger,2,42.049306,-87.677606
9,0x78dd1e,2019-09-22 00:10:00,2,fries,2,42.049306,-87.677606


In [12]:
df.dtypes

ticket_id           object
order_timestamp     object
location             int64
item_name           object
item_count           int64
lat                float64
long               float64
dtype: object

In [13]:
df.shape

(260645, 7)

In [50]:
df['item_name'].value_counts()

fries     100000
burger     76122
shake      46156
salad      38367
Name: item_name, dtype: int64

#### Possible data engineering:
1. Purchase time: breakfast, lunch, dinner, and somewhere in between
2. Location
3. Number of items purchased

Note:
At this point I personally think it is not reasonable to include month or date as a feature for clusting because clusting orders with month as a variable may not be interpreted as a behavior for customers within certain segment. (i.e. create a segment implying one group of customer like to place order in a particular month or date seems odd to me). 

In [14]:
df['ticket_id'].value_counts()

0x76906e    4
0x7aa21f    4
0x51dc46    4
0x5d5c55    4
0x12e981    4
           ..
0x63cc1f    2
0x6673ac    2
0x5938fe    2
0x1a9b9f    2
0x3cd19a    2
Name: ticket_id, Length: 100000, dtype: int64

In [27]:
# The dataset contains 100,000 purchase records
# Each records have various items been purchased

# Create a dataframe with each ticket id as a row
df_con = df[['ticket_id', 'location', 'order_timestamp']]

# Because the location and timestamp for each ticket will be the same, 
# other duplicates could be dropped
df_con.drop_duplicates(subset = 'ticket_id',inplace = True, ignore_index = True)

C:\Users\mabow\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [28]:
df_con

,ticket_id,location,order_timestamp
0,0x7901ee,7,2019-01-16 18:33:00
1,0x12b47f,3,2019-09-04 12:36:00
2,0x6d6979,6,2019-03-18 00:27:00
3,0x78dd1e,2,2019-09-22 00:10:00
4,0x4df8ab,8,2019-01-17 11:35:00
...,...,...,...
99995,0x804d8d,6,2019-01-08 00:28:00
99996,0x7b618c,9,2019-07-14 00:12:00
99997,0x75ab27,5,2019-04-03 11:31:00
99998,0x560060,6,2019-08-09 00:23:00


#### Purchase time 

In [30]:
# Convert datatype for 'order_timestamp' to datetime64 for further process
df_con['order_timestamp'] = pd.to_datetime(df_con['order_timestamp'])

C:\Users\mabow\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_con.dtypes

ticket_id                  object
location                    int64
order_timestamp    datetime64[ns]
dtype: object

In [38]:
# Check time period of this dataset
print ('The time period of this dataset are between ' + 
       str(df_con['order_timestamp'].min()) + ' and ' 
       + str(df_con['order_timestamp'].max()))


The time period of this dataset are between 2019-01-01 00:00:00 and 2019-12-30 23:59:00


In [39]:
# Get the hour of purchase for each order

df_con['order_hour'] = df_con['order_timestamp'].dt.hour

df_con.head()

C:\Users\mabow\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ticket_id,location,order_timestamp,order_hour
0,0x7901ee,7,2019-01-16 18:33:00,18
1,0x12b47f,3,2019-09-04 12:36:00,12
2,0x6d6979,6,2019-03-18 00:27:00,0
3,0x78dd1e,2,2019-09-22 00:10:00,0
4,0x4df8ab,8,2019-01-17 11:35:00,11


In [42]:
# Get the day of the week of purchase for each order
# May be used to create feature such as weekday or weekend 

df_con['day_of_week'] = df_con['order_timestamp'].dt.weekday

# Result is ranged from 0-6 instead of 1-7


C:\Users\mabow\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [48]:
# Based on the day_of_week, create weekday_order feature 
# to distinguish whether the purchase happended on weekday or weekend
df_con['weekday_order'] = np.where(df_con['day_of_week'] >= 5, 0, 1)

C:\Users\mabow\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
df_con

,ticket_id,location,order_timestamp,order_hour,day_of_week,weekday_order
0,0x7901ee,7,2019-01-16 18:33:00,18,2,1
1,0x12b47f,3,2019-09-04 12:36:00,12,2,1
2,0x6d6979,6,2019-03-18 00:27:00,0,0,1
3,0x78dd1e,2,2019-09-22 00:10:00,0,6,0
4,0x4df8ab,8,2019-01-17 11:35:00,11,3,1
...,...,...,...,...,...,...
99995,0x804d8d,6,2019-01-08 00:28:00,0,1,1
99996,0x7b618c,9,2019-07-14 00:12:00,0,6,0
99997,0x75ab27,5,2019-04-03 11:31:00,11,2,1
99998,0x560060,6,2019-08-09 00:23:00,0,4,1


In [52]:
# Possible item to purchase

items = list(set(df['item_name']))

items

['salad', 'fries', 'burger', 'shake']

In [55]:
# Create columns for each item
for i in items:
    df_con[i] = 0

df_con.head()

C:\Users\mabow\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ticket_id,location,order_timestamp,order_hour,day_of_week,weekday_order,salad,fries,burger,shake
0,0x7901ee,7,2019-01-16 18:33:00,18,2,1,0,0,0,0
1,0x12b47f,3,2019-09-04 12:36:00,12,2,1,0,0,0,0
2,0x6d6979,6,2019-03-18 00:27:00,0,0,1,0,0,0,0
3,0x78dd1e,2,2019-09-22 00:10:00,0,6,0,0,0,0,0
4,0x4df8ab,8,2019-01-17 11:35:00,11,3,1,0,0,0,0


In [59]:
# Compute # of item purchased for each order

for index,row in df_con.iterrows():
    tid = row['ticket_id']
    
    # Lookup the transaction of this particular order
    df_t = df.loc[df['ticket_id'] == tid] 
    
    # Initiate count for each item
    p_count = {'salad':0, 'fries':0, 'burger':0, 'shake': 0}
    
    # Iterate transaction of the order to update the count dict
    
    for ix, r in df_t.iterrows():
        item = r['item_name']
        quant = r['item_count']
        
        p_count[item] += quant
    
    # Remap the result to the concatenate dataframe
    
    for c in p_count.items():
        df_con.at[index, c[0]] = c[1]
    

This process takes a long time. It could be done faster using pd.merge if each item in every order only appear once at most. But I did not check whether this is the case.

In [60]:
df_con

,ticket_id,location,order_timestamp,order_hour,day_of_week,weekday_order,salad,fries,burger,shake
0,0x7901ee,7,2019-01-16 18:33:00,18,2,1,0,2,2,2
1,0x12b47f,3,2019-09-04 12:36:00,12,2,1,1,1,0,0
2,0x6d6979,6,2019-03-18 00:27:00,0,0,1,0,2,2,0
3,0x78dd1e,2,2019-09-22 00:10:00,0,6,0,0,2,2,2
4,0x4df8ab,8,2019-01-17 11:35:00,11,3,1,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...
99995,0x804d8d,6,2019-01-08 00:28:00,0,1,1,0,5,5,1
99996,0x7b618c,9,2019-07-14 00:12:00,0,6,0,0,4,4,2
99997,0x75ab27,5,2019-04-03 11:31:00,11,2,1,1,1,0,0
99998,0x560060,6,2019-08-09 00:23:00,0,4,1,0,2,2,0


In [61]:
# Export dataframe as csv file to save runtime

df_con.to_csv('ticket_info.csv', header = True, index = False)

## Please run the code from here after

In [62]:
df_con = pd.read_csv('ticket_info.csv')

Further thoughts:
1. Count number of types of items purchased per order (max at 4)?
2. Do clusting at different location? Or consider location as one variable